In [1]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset
from typing import Iterable, List
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import pandas as pd
from torchtext.data.metrics import bleu_score

##### Code built with the help of: https://pytorch.org/tutorials/beginner/translation_transformer.html

In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
FOLDER_PATH = '/drive/My Drive/Colab Notebooks/'

In [3]:
class IMDBDataset(Dataset):

    def __init__(self, filename, sentiment) -> None:
        super().__init__()
        data_df = pd.read_csv(filename)
        data_df['sentiment'] = data_df['sentiment'].apply(lambda x: 2 if x == 'pos' else 1)
        data_df['pair'] = list(zip(data_df['sentiment'], data_df['text']))

        if sentiment == 'pos':
            self.data = data_df['pair'][data_df['sentiment'] == 2].values.tolist()
        else:
            self.data = data_df['pair'][data_df['sentiment'] == 1].values.tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [4]:
SENTIMENT_POS = 'pos'
SENTIMENT_NEG = 'neg'

tokenizer = get_tokenizer('basic_english')
vocab_transform = {}

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

##### POSITIVE SENTIMENT VOCAB #####
train_iter_pos = IMDBDataset(f'{FOLDER_PATH}train.csv', SENTIMENT_POS)
vocab_transform[SENTIMENT_POS] = build_vocab_from_iterator(yield_tokens(train_iter_pos),
                                                min_freq=1,
                                                specials=special_symbols,
                                                special_first=True)
vocab_transform[SENTIMENT_POS].set_default_index(UNK_IDX)

##### NEGATIVE SENTIMENT VOCAB #####
train_iter_neg = IMDBDataset(f'{FOLDER_PATH}test.csv', SENTIMENT_NEG)
vocab_transform[SENTIMENT_NEG] = build_vocab_from_iterator(yield_tokens(train_iter_neg),
                                                min_freq=1,
                                                specials=special_symbols,
                                                special_first=True)
vocab_transform[SENTIMENT_NEG].set_default_index(UNK_IDX)

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [7]:
class Discriminator(nn.Module):

  def __init__(self, vocab_size, emb_size, dropout: float = 0.1):
    super(Discriminator, self).__init__()
    self.embedding = TokenEmbedding(vocab_size, emb_size)
    self.fc = nn.Linear(emb_size, 1)
    self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

  def forward(self, x):
    out = self.positional_encoding(self.embedding(x))
    out = nn.sigmoid(self.Linear(x))
    return out

In [8]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [9]:
def real_mse_loss(out, adverserial_weight=1):
    return torch.mean((out-1)**2)*adverserial_weight

def fake_mse_loss(out, adverserial_weight=1):
    return torch.mean(out**2)*adverserial_weight

def calc_cycle_loss(real_text, reconstructed_text, lambda_weight=1):
    return torch.mean(torch.abs(real_text - reconstructed_text))*lambda_weight

def calc_identity_loss(real_text, generated_text, identity_weight=1):
    return torch.mean(torch.abs(real_text - generated_text))*identity_weight

In [10]:
torch.manual_seed(0)

POS_VOCAB_SIZE = len(vocab_transform[SENTIMENT_POS])
NEG_VOCAB_SIZE = len(vocab_transform[SENTIMENT_NEG])
EMB_SIZE = 64
NHEAD = 8
FFN_HID_DIM = 64
BATCH_SIZE = 1
NUM_ENCODER_LAYERS = 1
NUM_DECODER_LAYERS = 1

G_POS_to_NEG = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, POS_VOCAB_SIZE, NEG_VOCAB_SIZE, FFN_HID_DIM)

G_NEG_to_POS = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, NEG_VOCAB_SIZE, POS_VOCAB_SIZE, FFN_HID_DIM)

D_NEG = Discriminator(NEG_VOCAB_SIZE, EMB_SIZE)
D_POS = Discriminator(POS_VOCAB_SIZE, EMB_SIZE)

for p in G_POS_to_NEG.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

for p in G_NEG_to_POS.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

for p in D_NEG.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

for p in D_POS.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

G_POS_to_NEG = G_POS_to_NEG.to(DEVICE)
G_NEG_to_POS = G_NEG_to_POS.to(DEVICE)
D_NEG = D_NEG.to(DEVICE)
D_POS = D_POS.to(DEVICE)

loss_fn_g = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
loss_fn_d = nn.BCELoss()
optimizer_g = torch.optim.Adam(list(G_POS_to_NEG.parameters()) + list(G_NEG_to_POS.parameters()), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
optimizer_d_pos = torch.optim.Adam(D_POS.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
optimizer_d_neg = torch.optim.Adam(D_NEG.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [11]:
pytorch_total_params = sum(p.numel() for p in G_NEG_to_POS.parameters() if p.requires_grad) \
  +sum(p.numel() for p in G_POS_to_NEG.parameters() if p.requires_grad)+sum(p.numel() for p in D_NEG.parameters() if p.requires_grad)+\
    sum(p.numel() for p in D_POS.parameters() if p.requires_grad)
pytorch_total_params

35375910

In [12]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SENTIMENT_POS, SENTIMENT_NEG]:
    text_transform[ln] = sequential_transforms(tokenizer,
                                               vocab_transform[ln],
                                               tensor_transform)


# function to collate data samples into batch tesors
def collate_fn(batch):
    text_batch = []
    for label, text in batch:
        if label == 1:
            text_batch.append(text_transform[SENTIMENT_NEG](text.rstrip("\n")))
        else:
            text_batch.append(text_transform[SENTIMENT_POS](text.rstrip("\n")))

    text_batch = pad_sequence(text_batch, padding_value=PAD_IDX)
    return text_batch

In [13]:
def train_epoch():
    G_POS_to_NEG.train()
    G_NEG_to_POS.train()

    losses = 0
    train_iter_pos = IMDBDataset(f'{FOLDER_PATH}train.csv', SENTIMENT_POS)
    train_iter_neg = IMDBDataset(f'{FOLDER_PATH}train.csv', SENTIMENT_NEG)
    train_pos_dataloader = DataLoader(train_iter_pos, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    train_neg_dataloader = DataLoader(train_iter_neg, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    
    for pos, neg in zip(train_pos_dataloader, train_neg_dataloader):
        pos = pos.to(DEVICE)
        neg = neg.to(DEVICE)

        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        optimizer_g.zero_grad()

        neg_input = neg[:-1, :]
        pos_input = pos[:-1, :]
        pos_mask1, neg_mask1, pos_padding_mask1, neg_padding_mask1 = create_mask(pos, neg_input)
        neg_mask2, pos_mask2, neg_padding_mask2, pos_padding_mask2 = create_mask(neg, pos_input)
        fake_neg = G_POS_to_NEG(pos, neg_input, pos_mask1, neg_mask1, pos_padding_mask1, neg_padding_mask1, pos_padding_mask1)
        fake_pos = G_NEG_to_POS(neg, pos_input, neg_mask2, pos_mask2, neg_padding_mask2, pos_padding_mask2, neg_padding_mask2)

        neg_out = neg[1:, :]
        generator_loss_pos = loss_fn_g(fake_neg.reshape(-1, fake_neg.shape[-1]), neg_out.reshape(-1))

        pos_out = pos[1:, :]
        generator_loss_neg = loss_fn_g(fake_pos.reshape(-1, fake_pos.shape[-1]), pos_out.reshape(-1))
        
        fake_neg_input = fake_neg[:-1, :]
        fake_pos_input = fake_pos[:-1, :]
        fake_pos_mask1, fake_neg_mask1, fake_pos_padding_mask1, fake_neg_padding_mask1 = create_mask(fake_pos, fake_neg_input)
        fake_neg_mask2, fake_pos_mask2, fake_neg_padding_mask2, fake_pos_padding_mask2 = create_mask(fake_neg, fake_pos_input)
        cycle_pos = G_NEG_to_POS(fake_neg, fake_pos_input, fake_neg_mask2, fake_pos_mask2, fake_neg_padding_mask2, fake_pos_padding_mask2, \
                                 fake_neg_padding_mask2)
        cycle_neg = G_POS_to_NEG(fake_pos, fake_neg_input, fake_pos_mask1, fake_neg_mask1, fake_pos_padding_mask1, fake_neg_padding_mask1, \
                                 fake_pos_padding_mask1)
        
        total_cycle_loss = calc_cycle_loss(pos, cycle_pos) + calc_cycle_loss(neg, cycle_neg)

        same_pos = G_NEG_to_POS(pos, neg_input, pos_mask2, neg_mask2, pos_padding_mask2, neg_padding_mask2, pos_padding_mask2)
        same_neg = G_POS_to_NEG(neg, pos_input, neg_mask1, pos_mask1, neg_padding_mask1, pos_padding_mask1, neg_padding_mask1)

        identity_loss = calc_identity_loss(pos, same_pos) + calc_identity_loss(neg, same_neg)

        total_generator_loss = generator_loss_pos.item() + generator_loss_neg.item() + total_cycle_loss + identity_loss
        total_generator_loss.backward()
        optimizer_g.step()

        # ================================================================== #
        #                        Train the discriminator                     #
        # ================================================================== #

        optimizer_d_pos.zero_grad()
        optimizer_d_neg.zero_grad()

        disc_real_pos = D_POS(pos)
        disc_real_neg = D_NEG(neg)

        disc_fake_pos = D_POS(fake_pos)
        disc_fake_neg = D_NEG(fake_neg)

        disc_pos_loss = loss_fn_d(disc_real_pos, disc_fake_pos)
        disc_neg_loss = loss_fn_d(disc_real_neg, disc_fake_neg)

        disc_pos_loss.backward()
        disc_neg_loss.backward()

        optimizer_d_pos.step()
        optimizer_d_neg.step()
        losses += total_generator_loss

    return losses / len(list(train_pos_dataloader))


def evaluate():
    G_POS_to_NEG.eval()
    G_NEG_to_POS.eval()
    losses = 0

    test_iter_pos = IMDBDataset(f'{FOLDER_PATH}test.csv', SENTIMENT_POS)
    test_iter_neg = IMDBDataset(f'{FOLDER_PATH}test.csv', SENTIMENT_NEG)
    test_pos_dataloader = DataLoader(test_iter_pos, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    test_neg_dataloader = DataLoader(test_iter_neg, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    
    for src, tgt in zip(test_pos_dataloader, test_neg_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(test_pos_dataloader))

In [14]:
from timeit import default_timer as timer
NUM_EPOCHS = 10

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch()
    end_time = timer()
    #val_loss = evaluate(G_NEG_to_POS, 'neg_to_pos')
    #print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f} "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


def transfer(model: torch.nn.Module, src_sentence: str, transfer_direction: str):
    model.eval()
    if transfer_direction == 'neg_to_pos':
        src = text_transform[SENTIMENT_NEG](src_sentence).view(-1, 1)
        num_tokens = src.shape[0]
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
        tgt_tokens = greedy_decode(
            model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
        return " ".join(vocab_transform[SENTIMENT_POS].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")
    else:
        src = text_transform[SENTIMENT_POS](src_sentence).view(-1, 1)
        num_tokens = src.shape[0]
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
        tgt_tokens = greedy_decode(
            model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
        return " ".join(vocab_transform[SENTIMENT_NEG].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

OutOfMemoryError: ignored

In [ ]:
print(transfer(G_NEG_to_POS, "This movie sucks.", 'neg_to_pos'))

In [ ]:
torch.save(G_NEG_to_POS.state_dict(), f'{FOLDER_PATH}G_NEG_to_POS.pkl')